## 📊 数据集准备

仅需要以以下文件结构将数据集放入 dataset_raw 目录即可。

```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav
```
对于每一个音频文件的名称并没有格式的限制(`000001.wav`~`999999.wav`之类的命名方式也是合法的)，不过文件类型必须是`wav`。

可以自定义说话人名称

```
dataset_raw
└───suijiSUI
    ├───1.wav
    ├───...
    └───25788785-20221210-200143-856_01_(Vocals)_0_0.wav
```

## 🛠️ 数据预处理

### 0. 音频切片

将音频切片至`5s - 15s`, 稍微长点也无伤大雅，实在太长可能会导致训练中途甚至预处理就爆显存

可以使用 [audio-slicer-GUI](https://github.com/flutydeer/audio-slicer)、[audio-slicer-CLI](https://github.com/openvpi/audio-slicer)

一般情况下只需调整其中的`Minimum Interval`，普通陈述素材通常保持默认即可，歌唱素材可以调整至`100`甚至`50`

切完之后手动删除过长过短的音频

### 1. 重采样至 44100Hz 单声道

```shell
python resample.py
```
#### 注意

虽然本项目拥有重采样、转换单声道与响度匹配的脚本 resample.py，但是默认的响度匹配是匹配到 0db。这可能会造成音质的受损。而 python 的响度匹配包 pyloudnorm 无法对电平进行压限，这会导致爆音。所以建议可以考虑使用专业声音处理软件如`adobe audition`等软件做响度匹配处理。若已经使用其他软件做响度匹配，可以在运行上述命令时添加`--skip_loudnorm`跳过响度匹配步骤。如：

```shell
python resample.py --skip_loudnorm
```

In [5]:
!python resample.py

CPU count: 22
./dataset_raw/syz
resampling: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00m 0:00:01


### 2. 自动划分训练集、验证集，以及自动生成配置文件

```shell
python preprocess_flist_config.py --speech_encoder vec768l12
```

speech_encoder 拥有以下选择

```
vec768l12
vec256l9
hubertsoft
whisper-ppg
whisper-ppg-large
cnhubertlarge
dphubert
wavlmbase+
```

如果省略 speech_encoder 参数，默认值为 vec768l12

In [6]:
!python preprocess_flist_config.py --speech_encoder vec768l12

  0%|                                                     | 0/1 [00:00<?, ?it/s]2025-09-03 10:57:23.187 | WARNING  | __main__:<module>:51 - 文件名./dataset/44k/syz/hey judy_voce_21_noreverb_0.wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
2025-09-03 10:57:23.187 | WARNING  | __main__:<module>:51 - 文件名./dataset/44k/syz/hey judy_voce_13_noreverb_0.wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
2025-09-03 10:57:23.187 | WARNING  | __main__:<module>:51 - 文件名./dataset/44k/syz/yujian_voce_(Vocals)_9_noreverb_0_1.wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
2025-09-03 10:57:23.187 | WARNING  | __main__:<module>:51 - 文件名./dataset/44k/syz/yujian_voce_(Vocals)_4_noreverb_0.wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
2025-09-03 10:57:23.187 | WARNING  | __main__:<module>:51 - 文件名./dataset/44k/syz/hey judy_voce_16_1_noreverb_0.wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
2025-09-03 10:57:23.188 | WARNING  | __main__:<module>:51 - 文件名./dataset/44k/syz/yujian_voce_(Vocals)_13_noreverb_0.wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
2025-09-03 10:57:23.188 | WARNING  | __main__:<module>:51

#### 此时可以在生成的 config.json 与 diffusion.yaml 修改部分参数

##### config.json

* `keep_ckpts`：训练时保留最后几个模型，`0`为保留所有，默认只保留最后`3`个

* `all_in_mem`：加载所有数据集到内存中，某些平台的硬盘 IO 过于低下、同时内存容量 **远大于** 数据集体积时可以启用

* `batch_size`：单次训练加载到 GPU 的数据量，调整到低于显存容量的大小即可

* `vocoder_name` : 选择一种声码器，默认为`nsf-hifigan`.

##### diffusion.yaml

* `cache_all_data`：加载所有数据集到内存中，某些平台的硬盘 IO 过于低下、同时内存容量 **远大于** 数据集体积时可以启用

* `duration`：训练时音频切片时长，可根据显存大小调整，**注意，该值必须小于训练集内音频的最短时间！**

* `batch_size`：单次训练加载到 GPU 的数据量，调整到低于显存容量的大小即可

* `timesteps` : 扩散模型总步数，默认为 1000.

* `k_step_max` : 训练时可仅训练`k_step_max`步扩散以节约训练时间，注意，该值必须小于`timesteps`，0 为训练整个扩散模型，**注意，如果不训练整个扩散模型将无法使用仅扩散模型推理！**

##### **声码器列表**

```
nsf-hifigan
nsf-snake-hifigan
```

### 3. 生成 hubert 与 f0

```shell
python preprocess_hubert_f0.py --f0_predictor dio
```

f0_predictor 拥有以下选择

```
crepe
dio
pm
harvest
rmvpe
fcpe
```

如果训练集过于嘈杂，请使用 crepe 处理 f0

如果省略 f0_predictor 参数，默认值为 dio

尚若需要浅扩散功能（可选），需要增加--use_diff 参数，比如

```shell
python preprocess_hubert_f0.py --f0_predictor dio --use_diff
```
**加速预处理**
如若您的数据集比较大，可以尝试添加`--num_processes`参数：
```shell
python preprocess_hubert_f0.py --f0_predictor dio --use_diff --num_processes 8
```
所有的Workers会被自动分配到多个线程上

执行完以上步骤后 dataset 目录便是预处理完成的数据，可以删除 dataset_raw 文件夹了


In [7]:
!python preprocess_hubert_f0.py --f0_predictor dio --use_diff --num_processes 8

vec768l12
2025-09-03 10:58:17.658 | INFO     | __main__:<module>:152 - Using device: 
2025-09-03 10:58:17.658 | INFO     | __main__:<module>:153 - Using SpeechEncoder: vec768l12
2025-09-03 10:58:17.658 | INFO     | __main__:<module>:154 - Using extractor: dio
2025-09-03 10:58:17.658 | INFO     | __main__:<module>:155 - Using diff Mode: True
use_diff
Loading Mel Extractor...
Loaded Mel Extractor.
  0%|                                                     | 0/8 [00:00<?, ?it/s]2025-09-03 10:58:19.596 | INFO     | __mp_main__:process_batch:107 - Loading speech encoder for content...
2025-09-03 10:58:19.600 | INFO     | __mp_main__:process_batch:107 - Loading speech encoder for content...
2025-09-03 10:58:19.601 | INFO     | __mp_main__:process_batch:107 - Loading speech encoder for content...
2025-09-03 10:58:19.612 | INFO     | __mp_main__:process_batch:113 - Rank 7 uses device cuda:0
2025-09-03 10:58:19.616 | INFO     | __mp_main__:process_batch:113 - Rank 8 uses device cuda:0
2025-09-03

## 🏋️‍ 训练

### 主模型训练

```shell
python train.py -c configs/config.json -m 44k
```

In [ ]:
!python train.py -c configs/config.json -m 44k

### 扩散模型（可选）

尚若需要浅扩散功能，需要训练扩散模型，扩散模型训练方法为：

```shell
python train_diff.py -c configs/diffusion.yaml
```

模型训练结束后，模型文件保存在`logs/44k`目录下，扩散模型在`logs/44k/diffusion`下

In [ ]:
!python train_diff.py -c configs/diffusion.yaml

## 🤖 推理

使用 [inference_main.py](inference_main.py)

```shell
# 例
python inference_main.py -m "logs/44k/G_30400.pth" -c "configs/config.json" -n "君の知らない物語-src.wav" -t 0 -s "nen"
```

必填项部分：
+ `-m` | `--model_path`：模型路径
+ `-c` | `--config_path`：配置文件路径
+ `-n` | `--clean_names`：wav 文件名列表，放在 raw 文件夹下
+ `-t` | `--trans`：音高调整，支持正负（半音）
+ `-s` | `--spk_list`：合成目标说话人名称
+ `-cl` | `--clip`：音频强制切片，默认 0 为自动切片，单位为秒/s

可选项部分：部分具体见下一节
+ `-lg` | `--linear_gradient`：两段音频切片的交叉淡入长度，如果强制切片后出现人声不连贯可调整该数值，如果连贯建议采用默认值 0，单位为秒
+ `-f0p` | `--f0_predictor`：选择 F0 预测器，可选择 crepe,pm,dio,harvest,rmvpe,fcpe, 默认为 pm（注意：crepe 为原 F0 使用均值滤波器）
+ `-a` | `--auto_predict_f0`：语音转换自动预测音高，转换歌声时不要打开这个会严重跑调
+ `-cm` | `--cluster_model_path`：聚类模型或特征检索索引路径，留空则自动设为各方案模型的默认路径，如果没有训练聚类或特征检索则随便填
+ `-cr` | `--cluster_infer_ratio`：聚类方案或特征检索占比，范围 0-1，若没有训练聚类模型或特征检索则默认 0 即可
+ `-eh` | `--enhance`：是否使用 NSF_HIFIGAN 增强器，该选项对部分训练集少的模型有一定的音质增强效果，但是对训练好的模型有反面效果，默认关闭
+ `-shd` | `--shallow_diffusion`：是否使用浅层扩散，使用后可解决一部分电音问题，默认关闭，该选项打开时，NSF_HIFIGAN 增强器将会被禁止
+ `-usm` | `--use_spk_mix`：是否使用角色融合/动态声线融合
+ `-lea` | `--loudness_envelope_adjustment`：输入源响度包络替换输出响度包络融合比例，越靠近 1 越使用输出响度包络
+ `-fr` | `--feature_retrieval`：是否使用特征检索，如果使用聚类模型将被禁用，且 cm 与 cr 参数将会变成特征检索的索引路径与混合比例

浅扩散设置：
+ `-dm` | `--diffusion_model_path`：扩散模型路径
+ `-dc` | `--diffusion_config_path`：扩散模型配置文件路径
+ `-ks` | `--k_step`：扩散步数，越大越接近扩散模型的结果，默认 100
+ `-od` | `--only_diffusion`：纯扩散模式，该模式不会加载 sovits 模型，以扩散模型推理
+ `-se` | `--second_encoding`：二次编码，浅扩散前会对原始音频进行二次编码，玄学选项，有时候效果好，有时候效果差


In [ ]:
!python inference_main.py -m "logs/20250903/G_10400.pth" -c "logs/20250903/config.json" -n "31948473856-1-192_vocals_noreverb.wav" -t 0 -s "syz" -wf wav -shd True

/root/anaconda3/envs/svc/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
load 
load model(s) from pretrain/checkpoint_best_legacy_500.pt
/root/anaconda3/envs/svc/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
#=====segment start, 16.24s======
jump empty segment
#=====segment start, 16.48s======
/root/sft/so-vits-svc/inference/infer_tool.py:270: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.

## 🗜️ 模型压缩

生成的模型含有继续训练所需的信息。如果确认不再训练，可以移除模型中此部分信息，得到约 1/3 大小的最终模型。

使用 [compress_model.py](compress_model.py)

```shell
# 例
python compress_model.py -c="models_backup/2025_07_21_13_19/config.json" -i="models_backup/2025_07_21_13_19/G_52000.pth" -o="trained/cxh_52000.pth"
```

In [14]:
!python compress_model.py -c="models_backup/2025_07_21_13_19/config.json" -i="models_backup/2025_07_21_13_19/G_52000.pth" -o="trained/cxh_52000.pth"

/root/anaconda3/envs/svc/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
